<a href="https://colab.research.google.com/github/swaminaathakrishnan/Cool_Route_prototype/blob/master/Cool_route_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚴 **CoolRide V4: Building Shadow + Time-Aware Routing**
### Project Overview

CoolRide V4 adds **building shadow modeling** and **time-aware routing** to the thermal comfort algorithm.

**New in V4:**
- 🏢 Building shadow calculation using sun position
- ⏰ Time-aware routing (different routes at different times of day)
- 🌡️ Combined tree + building shade analysis
- 📊 Improved cooling coverage (2x better than V3)

CoolRide is a TRL-6 prototype designed to route cyclists through thermally comfortable paths in Singapore. It utilizes real-time government weather data (NEA), building shadows, vegetation data, and predictive AI to mitigate Urban Heat Island (UHI) effects.

In [97]:
# ==========================================
# 🧱 MODULE 1: SYSTEM INITIALIZATION
# ==========================================
# Objective: Install geospatial libraries and set up the environment.
# dependencies: OSMnx (Maps), GeoPandas (Spatial Data), Scikit-Learn (AI).

!pip install osmnx simplekml geopandas shapely networkx requests scikit-learn -q

import osmnx as ox
import networkx as nx
import simplekml
import geopandas as gpd
import pandas as pd
import requests
import numpy as np
import os
import pickle
import time
from shapely.geometry import Point, LineString, box
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta

print("✅ System Initialized. Ready for V3 Execution.")

DEPRECATION: celery 4.2.0 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
✅ System Initialized. Ready for V3 Execution.


### ⚙️ Module 2: Configuration & Cloud Connection
Objective: Define the pilot zone and connect to the GitHub Data Lake. Logic: Instead of local files, we stream GeoJSON/CSV directly from the raw GitHub URLs. This allows the team to collaborate without sharing Drive folders.

In [98]:
# ==========================================
# ⚙️ MODULE 2: CONFIGURATION
# ==========================================

# # 1. PILOT ZONE
# PLACE_NAME = "Tampines, Singapore"
# START_POINT = (1.3533, 103.9452) # Tampines MRT
# END_POINT = (1.3598, 103.9351)   # Tampines Eco Green

#another test zone (that could show for demo)
# PLACE_NAME = "Marina Bay, Singapore"
# START_POINT = (1.2784, 103.8509)  # Marina Bay Sands
# END_POINT = (1.2866, 103.8545)     # Esplanade

# PLACE_NAME = "Bedok Reservoir, Singapore"
# START_POINT = (1.3390, 103.9230)  # Bedok North
# END_POINT = (1.3480, 103.9320)    # Bedok Reservoir Park

# PLACE_NAME = "Punggol, Singapore"
# START_POINT = (1.4040, 103.9020)  # Punggol MRT
# END_POINT = (1.4150, 103.9080)    # Waterway Point

# PLACE_NAME = "East Coast Park, Singapore"
# START_POINT = (1.3000, 103.9100)  # Bedok jetty area
# END_POINT = (1.3050, 103.9400)    # East Coast Park 


# PLACE_NAME = "Serangoon Gardens, Singapore"
# START_POINT = (1.3700, 103.8650)  # Serangoon Gardens
# END_POINT = (1.3800, 103.8750)    # Yio Chu Kang


# PLACE_NAME = "MacRitchie, Singapore"
# START_POINT = (1.3450, 103.8230)  # Lornie Road
# END_POINT = (1.3520, 103.8300)    # MacRitchie 

# 1. PILOT ZONE - Bedok Reservoir (Extended route)
# PLACE_NAME = "Bedok Reservoir, Singapore"
# START_POINT = (1.3360, 103.9180)  # Bedok Town Park (south)
# END_POINT = (1.3480, 103.9320)   

# PLACE_NAME = "Bedok Reservoir, Singapore"
# START_POINT = (1.3412, 103.9230)   # Forested PCN (west side)
# END_POINT   = (1.3438, 103.9285)   # Reservoir edge path

PLACE_NAME = "Bedok Reservoir, Singapore"

START_POINT = (1.3412, 103.9230)   # Forested PCN (west side)
END_POINT   = (1.3438, 103.9285)   # Reservoir edge path



# 2. TIME CONFIGURATION (NEW in V4!)
DEPARTURE_TIME = datetime(2024, 12, 21, 12, 0)  # Noon
#DEPARTURE_TIME = datetime.now()  # Or set specific time: datetime(2024, 6, 21, 14, 0)
print(f"   🕐 Departure Time: {DEPARTURE_TIME.strftime('%I:%M %p')}")

# 3. GITHUB DATA LAKE
GITHUB_USER = "swaminaathakrishnan"
REPO_NAME = "Cool_Route_prototype"
BASE_URL = f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/master/data/"

# File Links
PCN_URL = BASE_URL + "ParkConnectorLoop.geojson"
HAWKER_URL = BASE_URL + "HawkerCentresGEOJSON.geojson"
TREES_URL = BASE_URL + "trees.csv"

# 4. THERMAL WEIGHTS (Calibrated)
WEIGHT_PCN = 0.5          # Park connector discount
WEIGHT_TREE_SHADE = 0.6    # Tree shade discount
WEIGHT_BUILDING_SHADE = 0.7  # Building shade discount (NEW!)
WEIGHT_COMBINED = 0.4      # Trees + Buildings combined

# 5. SAFETY OVERRIDE
NEA_HEATWAVE_ALERT = False

print(f"✅ Configuration Loaded for {PLACE_NAME}")
print(f"   Data Lake: {GITHUB_USER}/{REPO_NAME}")

   🕐 Departure Time: 12:00 PM
✅ Configuration Loaded for Bedok Reservoir, Singapore
   Data Lake: swaminaathakrishnan/Cool_Route_prototype


### Module 3A: Sun Position & Building Shadow Engine 


Objective: Calculate real-time building shadows based on sun position and building heights.

**Physics:**
- Sun elevation angle determines shadow length
- Shadow direction is opposite of sun azimuth
- Taller buildings cast longer shadows (especially at low sun angles)

In [99]:

# MODULE 3A: SUN POSITION & BUILDING SHADOWS (V4)

def calculate_sun_position(latitude, longitude, timestamp):
    """
    Calculate sun elevation and azimuth for given location and time
    
    Returns: (elevation_degrees, azimuth_degrees)
    """
    import math
    day_of_year = timestamp.timetuple().tm_yday
    
    # Declination angle
    declination = 23.45 * math.sin(math.radians((360/365) * (day_of_year - 81)))
    
    # Hour angle
    hour = timestamp.hour + timestamp.minute / 60.0
    hour_angle = 15 * (hour - 12)
    
    # Sun elevation
    lat_rad = math.radians(latitude)
    dec_rad = math.radians(declination)
    ha_rad = math.radians(hour_angle)
    
    sin_elev = (math.sin(lat_rad) * math.sin(dec_rad) +
                math.cos(lat_rad) * math.cos(dec_rad) * math.cos(ha_rad))
    elevation = math.degrees(math.asin(max(-1, min(1, sin_elev))))
    
    # Sun azimuth
    cos_azim = ((math.sin(dec_rad) - math.sin(lat_rad) * sin_elev) /
                (math.cos(lat_rad) * math.cos(math.radians(elevation))))
    cos_azim = max(-1, min(1, cos_azim))
    azimuth = math.degrees(math.acos(cos_azim))
    
    if hour > 12:
        azimuth = 360 - azimuth
    
    return elevation, azimuth


def create_shadow_polygon(building_polygon, building_height, sun_elevation, sun_azimuth):
    """
    Create shadow polygon from building footprint
    
    Args:
        building_polygon: Shapely Polygon
        building_height: Height in meters  
        sun_elevation: Sun angle above horizon (degrees)
        sun_azimuth: Sun compass direction (degrees)
    
    Returns:
        Shadow polygon (Shapely)
    """
    import math
    from shapely.affinity import translate
    
    if sun_elevation <= 0:
        return None  # Night time
    
    # Shadow length = height / tan(elevation)
    shadow_length = building_height / math.tan(math.radians(sun_elevation))
    
    # Shadow direction (opposite of sun)
    shadow_direction = (sun_azimuth + 180) % 360
    
    # Calculate offset in meters
    shadow_offset_y = shadow_length * math.cos(math.radians(shadow_direction))
    shadow_offset_x = shadow_length * math.sin(math.radians(shadow_direction))
    
    # Get building centroid
    centroid = building_polygon.centroid
    lat, lon = centroid.y, centroid.x
    
    # Convert meters to degrees
    deg_per_meter_lat = 1 / 111000
    deg_per_meter_lon = 1 / (111000 * math.cos(math.radians(lat)))
    
    offset_lat = shadow_offset_y * deg_per_meter_lat
    offset_lon = shadow_offset_x * deg_per_meter_lon
    
    # Create shadow by translating building polygon
    shadow = translate(building_polygon, xoff=offset_lon, yoff=offset_lat)
    
    # Union with building for full coverage
    full_shadow = building_polygon.union(shadow).convex_hull
    
    return full_shadow

print("Sun Position & Building Shadow Functions Loaded")

Sun Position & Building Shadow Functions Loaded


### Module 3B: Enhanced Spatial Graph Engine

Objective: Build road network and overlay ALL cooling features.

**Layer 1:** Road Network (OSM)  
**Layer 2:** Park Connectors (PCN)  
**Layer 3:** Tree Canopy (SGTrees)  
**Layer 4:** Building Shadows (NEW! - Time-dependent)

Logic: Roads receive cumulative discounts based on shade coverage from multiple sources.

In [100]:
# ==========================================
# 🗺️ MODULE 3B: ENHANCED SPATIAL GRAPH ENGINE (V4)
# ==========================================

def generate_cool_routes():
    print(f"⏳ Downloading road network for {PLACE_NAME}...")

    # 1. GET GRAPH
    G = ox.graph_from_point(START_POINT, dist=2000, network_type='bike')
    nodes = ox.graph_to_gdfs(G, edges=False)
    miny, maxy = nodes.y.min(), nodes.y.max()
    minx, maxx = nodes.x.min(), nodes.x.max()
    print(f"   📐 Zone Limits: Lat[{miny:.4f}, {maxy:.4f}], Lon[{minx:.4f}, {maxx:.4f}]")

    # 2. LOAD PCN DATA
    print("⏳ Overlaying Park Connectors...")
    try:
        pcn_data = gpd.read_file(PCN_URL)
        if pcn_data.crs != "EPSG:4326": pcn_data = pcn_data.to_crs("EPSG:4326")
        try: pcn_union = pcn_data.geometry.union_all()
        except: pcn_union = pcn_data.geometry.unary_union
        print(f"   ✅ PCN Loaded")
    except Exception as e:
        print(f"   ⚠️ PCN Data missing: {e}")
        pcn_union = None

    # 3. LOAD TREE DATA
    print("⏳ Loading Tree Canopy Data...")
    trees_buffer = None
    try:
        # Use local file if exists, otherwise download
        local_file = "data/trees.csv"
        if not os.path.exists(local_file):
            local_file = "trees_downloaded.csv"
            lfs_url = f"https://github.com/{GITHUB_USER}/{REPO_NAME}/raw/master/data/trees.csv"
            print(f"   📥 Downloading trees...")
            response = requests.get(lfs_url, timeout=30, stream=True)
            response.raise_for_status()
            with open(local_file, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

        trees_df = pd.read_csv(local_file)
        trees_df.columns = [c.lower() for c in trees_df.columns]
        lat_col = 'latitude' if 'latitude' in trees_df.columns else 'lat'
        lng_col = 'longitude' if 'longitude' in trees_df.columns else 'lng'

        trees_df = trees_df[
            (trees_df[lat_col] >= miny) & (trees_df[lat_col] <= maxy) &
            (trees_df[lng_col] >= minx) & (trees_df[lng_col] <= maxx)
        ]

        print(f"   ✂️ Filtered: {len(trees_df)} trees in area")

        if len(trees_df) > 0:
            geometry = [Point(xy) for xy in zip(trees_df[lng_col], trees_df[lat_col])]
            trees_gdf = gpd.GeoDataFrame(trees_df, geometry=geometry, crs="EPSG:4326")
            trees_gdf_proj = trees_gdf.to_crs("EPSG:3414")
            trees_buffer_proj = trees_gdf_proj.geometry.buffer(10).union_all()
            trees_buffer = gpd.GeoSeries([trees_buffer_proj], crs="EPSG:3414").to_crs("EPSG:4326")[0]
            print(f"   ✅ Tree shade layer generated")
    except Exception as e:
        print(f"   ⚠️ Tree Data Error: {e}")

    # 4. LOAD BUILDINGS (NEW!)
    print("⏳ Loading Buildings from OpenStreetMap...")
    building_shadows = None
    try:
        buildings_gdf = ox.features_from_point(
            START_POINT,
            tags={'building': True},
            dist=2000
        )

        # Filter to polygons only
        buildings_gdf = buildings_gdf[buildings_gdf.geometry.type == 'Polygon']

        # Estimate heights
        def estimate_height(row):
            if 'height' in row and pd.notna(row['height']):
                try:
                    return float(str(row['height']).replace('m', ''))
                except:
                    pass
            if 'building:levels' in row and pd.notna(row['building:levels']):
                try:
                    return float(row['building:levels']) * 3
                except:
                    pass
            btype = str(row.get('building', '')).lower()
            if btype in ['commercial', 'office', 'retail']:
                return 50
            elif btype == 'residential':
                return 30
            return 15

        buildings_gdf['estimated_height'] = buildings_gdf.apply(estimate_height, axis=1)

        # Filter tall buildings (>20m)
        tall_buildings = buildings_gdf[buildings_gdf['estimated_height'] > 20].copy()

        print(f"   ✅ Found {len(tall_buildings)} tall buildings")

        # Calculate sun position
        sun_elev, sun_azim = calculate_sun_position(
            START_POINT[0],
            START_POINT[1],
            DEPARTURE_TIME
        )

        directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
        direction_idx = int((sun_azim + 22.5) / 45) % 8
        sun_dir = directions[direction_idx]

        print(f"   ☀️ Sun: {sun_elev:.1f}° elevation, {sun_azim:.1f}° azimuth ({sun_dir})")

        if sun_elev > 0:
            # Generate shadows
            shadow_polygons = []
            for idx, building in tall_buildings.iterrows():
                shadow = create_shadow_polygon(
                    building.geometry,
                    building['estimated_height'],
                    sun_elev,
                    sun_azim
                )
                if shadow:
                    shadow_polygons.append(shadow)

            # Combine all shadows
            from shapely.ops import unary_union
            building_shadows = unary_union(shadow_polygons)
            print(f"   ✅ Building shadow layer generated")
        else:
            print(f"   🌙 Night time - no building shadows")

    except Exception as e:
        print(f"   ⚠️ Building Data Error: {e}")

    # 5. LOAD SHELTERS
    shelters = []
    try:
        hawker_data = gpd.read_file(HAWKER_URL)
        hawker_data = hawker_data.cx[minx:maxx, miny:maxy]
        for _, row in hawker_data.iterrows():
            name = row.get('Name') or row.get('NAME') or 'Shelter'
            shelters.append((name, row.geometry.y, row.geometry.x))
    except:
        pass

    # 6. CALCULATE ENHANCED COOL COST
    print("⏳ Calculating Enhanced Thermal Costs...")
    shaded_by_buildings = 0
    shaded_by_trees = 0
    shaded_by_both = 0

    for u, v, k, data in G.edges(keys=True, data=True):
        if 'geometry' in data:
            edge_geom = data['geometry']
        else:
            edge_geom = LineString([(G.nodes[u]['x'], G.nodes[u]['y']),
                                   (G.nodes[v]['x'], G.nodes[v]['y'])])

        cost = data['length']
        is_pcn = False
        has_tree_shade = False
        has_building_shade = False

        # Check PCN
        if pcn_union and edge_geom.intersects(pcn_union):
            is_pcn = True
            cost *= WEIGHT_PCN

        # Check tree shade
        if trees_buffer and edge_geom.intersects(trees_buffer):
            has_tree_shade = True
            shaded_by_trees += 1

        # Check building shade (NEW!)
        if building_shadows and edge_geom.intersects(building_shadows):
            has_building_shade = True
            shaded_by_buildings += 1

        # Apply combined discounts
        if has_tree_shade and has_building_shade:
            cost *= WEIGHT_COMBINED  # Maximum cooling
            shaded_by_both += 1
            tag = "🏢🌳 Building + Tree Shade"
        elif has_building_shade:
            cost *= WEIGHT_BUILDING_SHADE
            tag = "🏢 Building Shadow"
        elif has_tree_shade:
            cost *= WEIGHT_TREE_SHADE
            tag = "🌳 Tree Shade"
        elif is_pcn:
            tag = "🌿 Park Connector"
        else:
            tag = "☀️ Exposed"

        data['cool_cost'] = cost
        data['tag'] = tag

    print(f"   📊 Shade Coverage:")
    print(f"      • Building shadows: {shaded_by_buildings} segments")
    print(f"      • Tree shade: {shaded_by_trees} segments")
    print(f"      • Combined: {shaded_by_both} segments")

    # 7. SOLVE
    orig = ox.distance.nearest_nodes(G, START_POINT[1], START_POINT[0])
    dest = ox.distance.nearest_nodes(G, END_POINT[1], END_POINT[0])

    try:
        r_fast = nx.shortest_path(G, orig, dest, weight='length')
        r_cool = nx.shortest_path(G, orig, dest, weight='cool_cost')
        return G, r_fast, r_cool, shelters
    except Exception as e:
        print(f"   ❌ Routing failed: {e}")
        return None, None, None, None

In [101]:
# ==========================================
# 🧠 MODULE 4: HISTORICAL AI ENGINE (V3)
# ==========================================
CACHE_FILE = "coolride_weather_memory.pkl"

def get_cache():
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, "rb") as f: return pickle.load(f)
        except: return {}
    return {}

def save_cache(data):
    with open(CACHE_FILE, "wb") as f: pickle.dump(data, f)

def fetch_historical_data(station_name, days_back=3):
    cache = get_cache()
    today_str = datetime.now().strftime("%Y-%m-%d")
    cache_key = f"{station_name}_{today_str}"

    if cache_key in cache and len(cache[cache_key]['values']) > 20:
        print(f"   ⚡ Memory Hit! Loaded {len(cache[cache_key]['values'])} points.")
        return cache[cache_key]['timestamps'], cache[cache_key]['values']

    print(f"   📡 Memory Miss. Analyzing last {days_back} days...")
    all_timestamps, all_values = [], []

    for i in range(days_back + 1):
        target_date = (datetime.now() - timedelta(days=i)).strftime("%Y-%m-%d")
        url = "https://api-open.data.gov.sg/v2/real-time/api/weather"
        params = {"api": "wbgt", "date": target_date}

        # PAGINATION LOOP
        while True:
            try:
                resp = requests.get(url, params=params, timeout=5)
                if resp.status_code != 200: break
                data = resp.json()
                if 'data' not in data: break

                for rec in data['data'].get('records', []):
                    dt = datetime.fromisoformat(rec['datetime'])
                    for r in rec['item']['readings']:
                        if r.get('station', {}).get('name') == station_name:
                            val = float(r.get('wbgt') or r.get('value'))
                            mins = dt.hour * 60 + dt.minute
                            now_mins = datetime.now().hour * 60 + datetime.now().minute
                            if abs(mins - now_mins) < 240: # 4 hour window
                                all_timestamps.append(mins)
                                all_values.append(val)

                token = data['data'].get('paginationToken')
                if token: params['paginationToken'] = token
                else: break
            except: break

    if len(all_values) > 20:
        cache[cache_key] = {'timestamps': all_timestamps, 'values': all_values}
        save_cache(cache)
        print(f"   💾 Learned & Saved {len(all_values)} thermal patterns.")

    return all_timestamps, all_values

def predict_trend(station_name, current_wbgt):
    timestamps, values = fetch_historical_data(station_name)
    if len(values) < 10: return current_wbgt, "Stable ➖", "Low Data"

    # Linear Regression
    model = LinearRegression()
    model.fit(np.array(timestamps).reshape(-1, 1), np.array(values))

    # Forecast
    now = datetime.now()
    fut_min = now.hour * 60 + now.minute + 15
    raw_pred = model.predict([[fut_min]])[0]

    # Physics Clamp
    delta = raw_pred - current_wbgt
    if abs(delta) > 0.5:
        final_pred = current_wbgt + (0.5 if delta > 0 else -0.5)
        note = "(Physics Clamped)"
    else:
        final_pred = raw_pred
        note = ""

    trend = "Rising 📈" if final_pred > current_wbgt + 0.1 else "Falling 📉" if final_pred < current_wbgt - 0.1 else "Stable ➖"
    return final_pred, trend, f"High {note}"

### 🚀 Module 5: Execution & Safe-Pace Recommendations

Objective: Synthesize map, weather, and AI data into a KML route. Upgrade:

* Govt Override: Checks NEA_HEATWAVE_ALERT.
* Safe Pacing: Suggests specific ride speeds and hydration intervals based on WBGT (ISO 7243 standards).

In [102]:
# ==========================================
# 🚀 MODULE 5: EXECUTION ENGINE (V3.9 - FINAL LABELS)
# ==========================================
import math
import os
import time
import requests
import simplekml

print("🚀 STARTING COOLRIDE ENGINE (WITH ACTIVE AI)...")

# --- HELPER: DYNAMIC SENSOR FINDER (FULL V2 LOGIC) ---
def get_nearest_wbgt_station(lat, lon):
    print("⏳ Connecting to NEA Official WBGT Sensor Network...")
    url = "https://api-open.data.gov.sg/v2/real-time/api/weather"
    try:
        resp = requests.get(url, params={"api": "wbgt"}, timeout=10)
        data = resp.json()
        readings = data['data']['records'][0]['item'].get('readings', [])

        closest_station = "Unknown"
        min_dist = float('inf')
        current_val = None

        for r in readings:
            try:
                loc = {}
                s_name = "Unknown"
                if 'location' in r: loc = r['location']
                elif 'station' in r and 'location' in r['station']: loc = r['station']['location']
                if 'station' in r: s_name = r['station'].get('name', 'Unknown')
                s_lat = float(loc.get('latitude', 0))
                s_lon = float(loc.get('longitude', loc.get('longtitude', 0)))
                if s_lat == 0 or s_lon == 0: continue

                val = r.get('wbgt') or r.get('value')
                if val is None: continue
                val = float(val)
                dist = math.sqrt((lat - s_lat)**2 + (lon - s_lon)**2)

                if dist < min_dist:
                    min_dist = dist
                    closest_station = s_name
                    current_val = val
            except: continue

        if current_val is None: return 30.0, "System Fallback"
        print(f"   📍 Nearest Sensor: {closest_station} (Dist: {min_dist*111:.2f} km)")
        return current_val, closest_station
    except Exception as e:
        print(f"   ⚠️ WBGT Sensor Fail: {e}. Using Default Safety Value.")
        return 30.0, "System Fallback"

# 1. GENERATE ROUTES
graph, r1, r2, shelters = generate_cool_routes()

if graph:
    # 2. GET WEATHER
    current_wbgt, station_name = get_nearest_wbgt_station(START_POINT[0], START_POINT[1])

    # 3. RUN AI
    pred_wbgt, trend, confidence = predict_trend(station_name, current_wbgt)
    effective_wbgt = max(current_wbgt, pred_wbgt)
    if NEA_HEATWAVE_ALERT: effective_wbgt = 35.0

    # 4. REPORT
    if effective_wbgt < 29: rec = "✅ Safe to Ride."
    elif effective_wbgt < 31: rec = "⚠️ CAUTION: Seek shade."
    else: rec = "🛑 HIGH RISK: Stop."

    print(f"\n📊 REPORT: {station_name}")
    print(f"   Current: {current_wbgt}°C | Forecast: {pred_wbgt:.1f}°C")

    # 5. EXPORT KML (CLEAN LABELS)
    kml = simplekml.Kml()

    def add_route(route, color, name, description):
        ls = kml.newlinestring(name=name)
        coords = []
        for u, v in zip(route[:-1], route[1:]):
            d = graph.get_edge_data(u, v)[0]
            if 'geometry' in d:
                xs, ys = d['geometry'].xy
                coords.extend(list(zip(xs, ys)))
            else:
                coords.append((graph.nodes[u]['x'], graph.nodes[u]['y']))
                coords.append((graph.nodes[v]['x'], graph.nodes[v]['y']))
        ls.coords = coords
        ls.style.linestyle.color = color
        ls.style.linestyle.width = 5
        ls.description = description

    # 🧠 FUZZY LOGIC + CLEAN LABELS
    def check_similarity(route_a, route_b):
        set_a = set(route_a)
        set_b = set(route_b)
        intersection = len(set_a.intersection(set_b))
        union = len(set_a.union(set_b))
        return intersection / union

    sim_score = check_similarity(r1, r2)
    print(f"   🔍 Route Similarity Score: {sim_score*100:.1f}%")

    if sim_score > 0.90:  # If >90% similar, merge them
        print("   💡 Insight: Routes are effectively identical (Merged).")
        # MERGED LABEL
        add_route(r2, simplekml.Color.green, "🌟 Recommended Route",
                  f"<b>Smart Choice</b><br>The fastest path is also the coolest.<br>No detour needed.<br><br>Temp: {effective_wbgt:.1f}°C")
    else:
        print("   💡 Insight: A distinct cooler detour exists.")
        # DIVERGENT LABELS
        add_route(r1, simplekml.Color.red, "⚡ Fastest Route (Exposed)",
                  f"<b>Direct Path</b><br>Shortest time, but higher heat exposure.<br><br>Temp: {effective_wbgt:.1f}°C")
        add_route(r2, simplekml.Color.green, "🌿 Cool Route (Shaded)",
                  f"<b>Shaded Detour</b><br>Maximized tree canopy coverage.<br>Lower heat stress.<br><br>Temp: {effective_wbgt:.1f}°C")

    # Add Shelters
    if shelters:
        for name, lat, lon in shelters:
            p = kml.newpoint(name=f"🧊 {name}", coords=[(lon, lat)])
            p.style.iconstyle.icon.href = 'http://googleusercontent.com/maps.google.com/mapfiles/kml/shapes/snowflake_simple.png'

    # 6. SAVE
    if not os.path.exists('output'): os.makedirs('output')
    constant_filename = "output/latest_route.kml"
    kml.save(constant_filename)

    print(f"\n🎉 SUCCESS! Download '{constant_filename}'")
else:
    print("❌ Critical Error: Route Generation Failed.")

🚀 STARTING COOLRIDE ENGINE (WITH ACTIVE AI)...
⏳ Downloading road network for Bedok Reservoir, Singapore...
   📐 Zone Limits: Lat[1.3232, 1.3592], Lon[103.9051, 103.9410]
⏳ Overlaying Park Connectors...
   ✅ PCN Loaded
⏳ Loading Tree Canopy Data...
   ✂️ Filtered: 22556 trees in area
   ✅ Tree shade layer generated
⏳ Loading Buildings from OpenStreetMap...
   ✅ Found 1938 tall buildings
   ☀️ Sun: 65.2° elevation, 180.0° azimuth (S)
   ✅ Building shadow layer generated
⏳ Calculating Enhanced Thermal Costs...
   📊 Shade Coverage:
      • Building shadows: 1113 segments
      • Tree shade: 2425 segments
      • Combined: 337 segments
⏳ Connecting to NEA Official WBGT Sensor Network...
   📍 Nearest Sensor: Bedok North Street 2 (Dist: 2.52 km)
   ⚡ Memory Hit! Loaded 111 points.

📊 REPORT: Bedok North Street 2
   Current: 28.7°C | Forecast: 28.2°C
   🔍 Route Similarity Score: 92.9%
   💡 Insight: Routes are effectively identical (Merged).

🎉 SUCCESS! Download 'output/latest_route.kml'
